# Цель
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE (Mean Absolute Percentage Error)

В рамках этого проекта мы использовали:

* Ноутбук, через который парсили https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-data
* Спарсенный датасет https://www.kaggle.com/juliadeinego/data-car-sales
* Ноутбук, в котором провели EDA&FE https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parser-eda
* Ноутбук, в котором провели обучение https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-ml

# Задачи

1. Собрать данные по предложениям о продаже автомобилей с auto.ru
2. Сохранить порционно в файлы для дальнейшей работы.

In [1]:
#!pip install BeautifulSoup4

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import glob
import json
import concurrent.futures
import time
import datetime

# Функции для парсинга

In [43]:
# функция получения страницы по адресу


def get_page(url):
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 'Cookie': '_csrf_token=e06ec3b4cac26954499a25d7a651baca0cb09dfe3f13be90; autoru_gdpr=1; autoruuid=g6066406e2k99ttmc5t5ladjk4aif7r7.c902643577af5e4f51d7218fc1fddd7f; suid=d3ae225d6b7f58d77e0faf5501c45f89.76ffd8381b3c3f209a7ac40e8e5ef0ba; yuidlt=1; yandexuid=9266985931608033188; my=YwA=; gdpr=0; _ym_uid=1617313903102348004; listing_view_session={}; autoru_sid=a:g6066406e2k99ttmc5t5ladjk4aif7r7.c902643577af5e4f51d7218fc1fddd7f|1617918702051.604800.nqu_KnagaSJ5L8T4V_-u9g.bvcge8gBjTOxGZ3-ZSojsTkz1tOVzf47pl3q0KcJvMU; X-Vertis-DC=vla; _ym_isad=2; gids=225; listing_view={"output_type":"table","version":1}; autoru-visits-count=2; autoru-visits-session-unexpired=1; from=other; _ym_d=1617913472; from_lifetime=1617913472696; cycada=/tGSgvkfuQovrGbiPr9JRwTgF3/yZiJBexlRLUSI20U='
    #           }
    response = requests.get(url)#, headers=headers)
    response.encoding = 'utf8'
    return BeautifulSoup(response.text, 'html.parser')

In [44]:
# функция получения списка марок автомобилей


def get_brand_links():
    url = f'https://auto.ru'
    soup_main = get_page(url)

    all_brand_links = []

    for brand in soup_main.find_all('a', class_='IndexMarks__item'):
        all_brand_links.append(brand.get('href').replace('all', 'used'))

    return all_brand_links

In [45]:
# функция проверки числа объявлений - находит кнопку "Показать **** предложения" и выдергивает число
# далее количество ссылок должно совпадать с эти числом


def check_number(url):
    strr = 'ButtonWithLoader__content'
    page = get_page(url)
    cntt = page.find(class_=strr).text.split(' ')[1].split('\xa0') # пример Показать 37 553 предложения
    if len(cntt[1]) > 3: 
        cnt = int(cntt[0])
    else:
        cnt = int(cntt[0]+cntt[1])
    return cnt

In [46]:
# функция получения конечного списка адресов предложений со страницы со списком (https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=2) 

def get_page_car_links(page_url):
    page = get_page(page_url)
    if page == None:
        return []
    #all_link = page.find_all(class_='ListingItemTitle-module__link')
    all_link = page.find_all('a',class_='Link ListingItemTitle__link')
    links = []
    for link in all_link:
        links.append(link['href'])
    return links

In [47]:
# функция получения списка адресов
# https://www.digitalocean.com/community/tutorials/how-to-use-threadpoolexecutor-in-python-3-ru

def get_car_links(url):
    try:
        strr = 'Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination__page'# strr = 'ListingPagination-module__page'
        # strr = 'ListingPagination-module__page'
        max_page_num = int(get_page(url).find_all('a', class_=strr)[-1].text)    
    except:
        max_page_num = 0
        
    links = []
    
    page_urls = [url] # ссылка на 1 страницу (в том числе единственную)
    if max_page_num > 1: 
        page_urls += [url + '?page=' + str(i) for i in range(2, max_page_num+1)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for url in page_urls:
            futures.append(executor.submit(get_page_car_links, page_url=url))
        for future in concurrent.futures.as_completed(futures):
            links += future.result()
    return links

In [48]:
# функция получения списка ссылок на все объявления по заданной марке, напрпимер 'https://auto.ru/cars/vaz/used/'


def get_links(brand_links):
    car_body = ['sedan', 'hatchback', 'allroad', 'wagon',
                'coupe', 'minivan', 'pickup', 'limousine', 'van', 'cabrio']
    car_links = []
    for link in brand_links:
        #print('s0', link)
        cnt = check_number(link)
        if cnt <= 3700: # если предложений меньше 3700 выкачиваем ссылки без перелистывания года и типа кузова
            car_links += get_car_links(link)
            #print('s1', link)
        else:  #если предложений больше 3700 выкачиваем ссылки перелистывая года
            for year in range(2000, 2022): 
                if year > 2000: # если больше 2000
                    aa = link.split('/')
                    url_1 = '/'.join(aa[:5])+'/'+str(year)+'-year/used/' #https://auto.ru/cars/vaz/2001-year/used/
                    #! при выкачке по региону стовиться aa[:6]
                    #print('s2', url_1)
                else: # если меньше 2000
                    url_1 = link+'?year_to=2000' #https://auto.ru/cars/vaz/used/?year_to=2000
                    #print('s3', url_1)
                cnt = check_number(url_1)
                print(cnt)
                if cnt <= 3700:
                    car_links += get_car_links(url_1)
                    #print('s4', car_links)
                else: #при выборке по одному региону (даже москве) в эту ветку не заходили, так как по году выпуска предложений меньше
                    for cb in car_body:
                        if year > 2000:
                            url_2 = url_1+f'body-{cb}/'
                            print('s5', url_2)
                        else:
                            url_2 = link+f'body-{cb}/?year_to=2000'
                        #print('s6', url_2)
                        car_links += get_car_links(url_2)
    print(brand_links, car_links)
    return car_links

    '''   поля тестовой базы     
    0   bodyType              34686 non-null  object  --
    1   brand                 34686 non-null  object --
    2   car_url               34686 non-null  object
    3   color                 34686 non-null  object --
    4   complectation_dict    6418 non-null   object нету в тест базе 81 % null, не заморачиваюсь на заполнение
    5   description           34686 non-null  object --
    6   engineDisplacement    34686 non-null  object --
    7   enginePower           34686 non-null  object --
    8   equipment_dict        24690 non-null  object -- не могу найти схожий по наполнению
    9   fuelType              34686 non-null  object --
    10  image                 34686 non-null  object --
    11  mileage               34686 non-null  int64 --
    12  modelDate             34686 non-null  int64 --нету
    13  model_info            34686 non-null  object --нету
    14  model_name            34686 non-null  object --
    15  name                  34686 non-null  object --
    16  numberOfDoors         34686 non-null  int64 --
    17  parsing_unixtime      34686 non-null  int64 --
    18  priceCurrency         34686 non-null  object -- нету 1 значение не паримся
    19  productionDate        34686 non-null  int64 --
    20  sell_id               34686 non-null  int64 --
    21  super_gen             34686 non-null  object --нету
    22  vehicleConfiguration  34686 non-null  object --нету
    23  vehicleTransmission   34686 non-null  object --
    24  vendor                34686 non-null  object --нету
    25  Владельцы             34686 non-null  object --
    26  Владение              11995 non-null  object --нету
    27  ПТС                   34685 non-null  object --
    28  Привод                34686 non-null  object --
    29  Руль                  34686 non-null  object --
    30  Состояние             34686 non-null  object --
    31  Таможня               34686 non-null  object --
    '''

In [53]:
# функция получения данных с одной страницы

def get_car_info(car_link):
    soup_car = get_page(car_link)
    car_info = {}
    cls_str = 'CardInfoRow_'
    span_str = 'CardInfoRow__cell'
    sd = 'sale-data-attributes'
    if (soup_car.find('div', class_='CardSold') is None)&(soup_car.find('div', title='Идентификатор объявления') is not None):
        # если нет ифнормации о продаже авто и есть само объявление (не удалено)
        try:
            car_info['datetime'] = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=6))) #add columns
            car_info['parsing_unixtime'] = int(time.time()) # is
#        car_info['region'] = soup_car.find('span', class_='MetroListPlace__regionName MetroListPlace_nbsp').text
#        if soup_car.find('div', class_='CardSellerNamePlace__name'):
#            car_info['sellerName'] = soup_car.find('div', class_='CardSellerNamePlace__name').text
#        elif soup_car.find('a', class_='CardSellerNamePlace__name_dealer'):
#            car_info['dealerName'] = soup_car.find('a', class_='CardSellerNamePlace__name_dealer').text
            car_info['dateAdvertisement'] = soup_car.find('div', class_='CardHead__infoItem CardHead__creationDate').text #add columns
            car_info['sell_id'] = soup_car.find('div', title='Идентификатор объявления').text # is
            car_info['car_url'] = car_link  # is
            car_info['price'] = soup_car.find('span', class_='OfferPriceCaption__price').text.replace('\xa0','')
            
            try:
                car_info['description'] = soup_car.find('div', class_='CardDescription__textInner').text # is
            except:
                car_info['description'] = None
                
            try:    
                car_info['image'] = 'https:' + soup_car.find('img', class_='ImageGalleryDesktop__image').get('src') 
            except:
                car_info['image'] = None   
            
            car_info['bodyType'] = soup_car.find('li', class_=cls_str+'bodytype').find('a').text # is
            car_info['color'] = soup_car.find('li', class_=cls_str+'color').find('a').text  # is
            
#----------------  
            try:
                car_info['engine'] = soup_car.find('li', class_=cls_str+'engine').find('div').text.replace('\xa0','')
                car_info['engineDisplacement'], car_info['enginePower'], car_info['fuelType'] = car_info['engine'].split('/') # is
            except:
                try:
                    car_info['engine'] = soup_car.find('li', class_=cls_str+'engine').find_all(class_ = 'CardInfoRow__cell')[1].text.replace('\xa0','')
                    car_info['engineDisplacement'] = 0
                    car_info['enginePower'] = car_info['engine'].split('/')[0] 
                    car_info['fuelType'] = car_info['engine'].split('/')[2] # is
                except:
                    car_info['engine'] = None
                    car_info['engineDisplacement'] = None
                    car_info['enginePower'] = None
                    car_info['fuelType'] = None
#---------------- 
            
            car_info['mileage'] = soup_car.find('li', class_=cls_str+'kmAge').find_all('span')[1].text.replace('\xa0','') # is
            car_info['productionDate'] = soup_car.find('li', class_=cls_str+'year').find_all('span')[1].text # is
            car_info['vehicleTransmission'] = soup_car.find('li', class_=cls_str+'transmission').find_all('span')[1].text # is
            try:
                car_info['Владельцы'] = soup_car.find('li', class_=cls_str+'ownersCount').find_all('span')[1].text # is
            except:
                car_info['Владельцы'] = None
                
            try:    
                car_info['ПТС'] = soup_car.find('li', class_=cls_str+'pts').find_all('span')[1].text # is
            except:
                car_info['ПТС'] = None
                
            car_info['Привод'] = soup_car.find('li', class_=cls_str+'drive').find_all('span')[1].text # is
            car_info['Руль'] = soup_car.find('li', class_=cls_str+'wheel').find_all('span')[1].text # is
            car_info['Состояние'] = soup_car.find('li', class_=cls_str+'state').find_all('span')[1].text # is
            car_info['Таможня'] = soup_car.find('li', class_=cls_str+'customs').find_all('span')[1].text # is
#        car_info['nview'] = soup_car.find_all('div', class_= 'CardHead__infoItem')[1].text.split(' ')[0]
            car_info['model_name_full'] = soup_car.find('h1').text
       
            soup_name = soup_car.find('div', class_='CardBreadcrumbs').find_all('a')
            car_info['card_type'] = soup_name[1].text
            car_info['brand'] = soup_name[2].text  # is
            car_info['model_name'] = soup_name[3].text
            car_info['model_name2'] = soup_name[4].text
            car_info['name'] = soup_name[6].text
#        car_info['equipment_dict'] = sale_data = json.loads(soup_car.find(id=sd)['data-bem'])[sd]
      
            url_2 = soup_car.find('a', class_='SpoilerLink')['href']
            pag = get_page(url_2)
            car_info['confDict'] = json.loads(pag.find(id=sd)['data-bem'])[sd]
        
        # полезли за деталями в каталог модели
            link_catalog_model = soup_car.find('a', class_ = 'Link SpoilerLink CardCatalogLink SpoilerLink_type_default')['href']
            car_catalog = get_page(link_catalog_model)
            car_info['numberOfDoors'] = car_catalog.find_all('dl', class_ = 'list-values clearfix')[0].find_all('dd', class_ = 'list-values__value')[2].text
        except:
            print ('Не найден какой тэг:'+ car_link)
    else:
        car_info = {}

    return car_info

In [50]:
# функция получения данных по дипазону адресов
# использование пула потоков для асинхронного выполнения HTTP запросов
# пример отсюда https://docs-python.ru/standart-library/modul-concurrent-futures-python/funktsija-threadpoolexecutor-modulja-concurrent-futures/

def get_data(urls):
    data = []
    # используем оператор `with` для обеспечения быстрой очистки потоков
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # загружаем и отмечаем каждый будущий результат своим URL-адресом
        futures_of_url = {executor.submit(get_car_info, url): url for url in urls} # Executor.submit() планирует выполнение потоков
        for future in concurrent.futures.as_completed(futures_of_url):
    # Функция as_completed ожидает каждого вызова Future get_car_info(url) для выполнения, чтобы дать future.result() 
            url = futures_of_url[future]
            try:
                data.append(future.result())
                time.sleep(0.4) 
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
    return data

## 1 часть
## записываем все ссылки в несколько файлов 

In [11]:
#br = get_brand_links() # получили список ссылок на перечень брендов
# не пользовалась так как достала список ссылок на перечень брендов вручную 
br = pd.read_csv('./data/links/12_brand_links.csv', sep=';', header=None) # len(br) = 131 по москве
br

,0,1
0,bmw,https://auto.ru/cars/bmw/used/
1,volkswagen,https://auto.ru/cars/volkswagen/used/
2,nissan,https://auto.ru/cars/nissan/used/
3,mercedes,https://auto.ru/cars/mercedes/used/
4,toyota,https://auto.ru/cars/toyota/used/
5,audi,https://auto.ru/cars/audi/used/
6,mitsubishi,https://auto.ru/cars/mitsubishi/used/
7,skoda,https://auto.ru/cars/skoda/used/
8,volvo,https://auto.ru/cars/volvo/used/
9,honda,https://auto.ru/cars/honda/used/


In [11]:
import traceback

link_dict = {}
number_portion = 0 #номер порции для записи в файл (будет порядка 5 порций)
start_point = 0 # начало из списка брэндов
end_point = len(br) #len(br) # конец из списка брэндов, для полной выгрузки использовать len(br)

for i in range(start_point, end_point): 
    brand = br[0][i] # наименование бренда
    link = br[1][i] #ссылка на бренд
    number_link = check_number(link) # количество объявлений, должно совпасть с количеством ссылок
    print(brand, link, number_link)
    threaded_start = time.time()
    
    try:
        link_dict[brand] = get_links([link])
    except Exception:
        traceback.print_exc()
        link_dict[brand] = []
        
    print("Load time:", time.time() - threaded_start, "; Brand:", brand, "; Кол-во ссылок:", number_link,"; Скачано:", len(link_dict[brand]))
    
    if ((i%12 == 0)&(i > 0))|(i == end_point - 1):  # разбиваем порции по 20 брендов (сливать части в блокноте, ексель ставит перенос строки) 
        number_portion += 1
        df = pd.DataFrame(link_dict.items()) 
        # запись в файл
        file_name = './data/'+ str(number_portion)+'_cars_links_by_brand'+'.csv'
        df.to_csv(file_name, sep=';', encoding='utf-8', index=False)    
        link_dict = {}

bmw https://auto.ru/cars/bmw/used/ 11622


KeyboardInterrupt: 

In [22]:
 df

,0,1
0,bmw,[https://auto.ru/cars/used/sale/bmw/5er/110646...


In [141]:
''' проверочный блок по одной ссылке из br (ссылка по одному бренду)
link =  'https://auto.ru/moskva/cars/bmw/used/'
print(link, check_number(link))
threaded_start = time.time()
brand = link.split('/')[5]
res = {}
try:
    print('0', brand)
    if not len(res_dict[brand])==0: # считает длину ссылки
        #res_dict[brand] = get_links([link])
        res[brand] = get_links([link])
except:
    #res_dict[brand]=[]
    res[brand]=[]
    print('2')
print("Load time:", time.time() - threaded_start, len(res[brand]))
'''

https://auto.ru/moskva/cars/bmw/used/ 4178
0 bmw
s0 https://auto.ru/moskva/cars/bmw/used/
s3 https://auto.ru/moskva/cars/bmw/used/?year_to=2000
314
['https://auto.ru/moskva/cars/bmw/used/?year_to=2000', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=2', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=3', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=4', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=5', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=6', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=7', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=8', 'https://auto.ru/moskva/cars/bmw/used/?year_to=2000?page=9']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55

37
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bmw/5er/1106468651-4d109b70/', 'https://auto.ru/cars/used/sale/bmw/5er/1106353146-151e71e2/', 'https://auto.ru/cars/used/sale/bmw/5er/1106281036-377370e7/', 'https://auto.ru/cars/used/sale/bmw/5er/1106426144-fedff5b2/', 'https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/', 'https://auto.ru/cars/used/sale/bmw/7er/1076222725-8d8d1b/', 'https://auto.ru/cars/used/sale/bmw/7er/1101754513-4305c66a/', 'https://auto.ru/cars/used/sale/bmw/5er/1103260263-206

49
['https://auto.ru/moskva/cars/bmw/2002-year/used/', 'https://auto.ru/moskva/cars/bmw/2002-year/used/?page=2']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bmw/5er/1106468651-4d109b70/', 'https://auto.ru/cars/used/sale/bmw/5er/1106353146-151e71e2/', 'https://auto.ru/cars/used/sale/bmw/5er/1106281036-377370e7/', 'https://auto.ru/cars/used/sale/bmw/5er/1106426144-fedff5b2/', 'https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/', 'https://auto.ru/cars/used/sale/bmw/7er/1076222725-8d8d1b/', 'https:

53
['https://auto.ru/moskva/cars/bmw/2003-year/used/', 'https://auto.ru/moskva/cars/bmw/2003-year/used/?page=2']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bmw/5er/1106468651-4d109b70/', 'https://auto.ru/cars/used/sale/bmw/5er/1106353146-151e71e2/', 'https://auto.ru/cars/used/sale/bmw/5er/1106281036-377370e7/', 'https://auto.ru/cars/used/sale/bmw/5er/1106426144-fedff5b2/', 'https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/', 'https://auto.ru/cars/used/sale/bmw/7er/1076222725-8d8d1b/', 'https:

56
['https://auto.ru/moskva/cars/bmw/2004-year/used/', 'https://auto.ru/moskva/cars/bmw/2004-year/used/?page=2']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bmw/5er/1106468651-4d109b70/', 'https://auto.ru/cars/used/sale/bmw/5er/1106353146-151e71e2/', 'https://auto.ru/cars/used/sale/bmw/5er/1106281036-377370e7/', 'https://auto.ru/cars/used/sale/bmw/5er/1106426144-fedff5b2/', 'https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/', 'https://auto.ru/cars/used/sale/bmw/7er/1076222725-8d8d1b/', 'https:

...

278
['https://auto.ru/moskva/cars/bmw/2017-year/used/', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=2', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=3', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=4', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=5', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=6', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=7', 'https://auto.ru/moskva/cars/bmw/2017-year/used/?page=8']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bm

353
['https://auto.ru/moskva/cars/bmw/2018-year/used/', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=2', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=3', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=4', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=5', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=6', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=7', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=8', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=9', 'https://auto.ru/moskva/cars/bmw/2018-year/used/?page=10']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er

276
['https://auto.ru/moskva/cars/bmw/2019-year/used/', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=2', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=3', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=4', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=5', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=6', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=7', 'https://auto.ru/moskva/cars/bmw/2019-year/used/?page=8']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bm

292
['https://auto.ru/moskva/cars/bmw/2020-year/used/', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=2', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=3', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=4', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=5', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=6', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=7', 'https://auto.ru/moskva/cars/bmw/2020-year/used/?page=8']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bm

166
['https://auto.ru/moskva/cars/bmw/2021-year/used/', 'https://auto.ru/moskva/cars/bmw/2021-year/used/?page=2', 'https://auto.ru/moskva/cars/bmw/2021-year/used/?page=3', 'https://auto.ru/moskva/cars/bmw/2021-year/used/?page=4', 'https://auto.ru/moskva/cars/bmw/2021-year/used/?page=5']
s4 ['https://auto.ru/cars/used/sale/bmw/5er/1106414597-b135c07b/', 'https://auto.ru/cars/used/sale/bmw/7er/1106360387-b3c299fd/', 'https://auto.ru/cars/used/sale/bmw/5er/1106428778-10df2356/', 'https://auto.ru/cars/used/sale/bmw/3er/1106196182-4e46d84d/', 'https://auto.ru/cars/used/sale/bmw/321/1106015754-55d234e5/', 'https://auto.ru/cars/used/sale/bmw/5er/1100491660-a94d60c3/', 'https://auto.ru/cars/used/sale/bmw/5er/1106282261-5e255459/', 'https://auto.ru/cars/used/sale/bmw/3er/1080427174-783f/', 'https://auto.ru/cars/used/sale/bmw/5er/1106468651-4d109b70/', 'https://auto.ru/cars/used/sale/bmw/5er/1106353146-151e71e2/', 'https://auto.ru/cars/used/sale/bmw/5er/1106281036-377370e7/', 'https://auto.ru/ca

## 2 часть.
### 2.1 Загружаем все ссылки из файлов в dataFrame

In [12]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 100)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', 100)
 
# Сброс ограничений на количество символов в записи
#pd.set_option('display.max_colwidth', None)

In [10]:
db_link = pd.read_csv('./data/links/12_brand_links_all.csv', sep=';') 
db_link

,0,1
0,bmw,['https://auto.ru/cars/used/sale/bmw/5er/11064...
1,volkswagen,['https://auto.ru/cars/used/sale/volkswagen/je...
2,nissan,['https://auto.ru/cars/used/sale/nissan/primer...
3,mercedes,['https://auto.ru/cars/used/sale/mercedes/w124...
4,toyota,['https://auto.ru/cars/used/sale/toyota/echo/1...
5,audi,['https://auto.ru/cars/used/sale/audi/100/1106...
6,mitsubishi,['https://auto.ru/cars/used/sale/mitsubishi/as...
7,skoda,['https://auto.ru/cars/used/sale/skoda/octavia...
8,volvo,['https://auto.ru/cars/used/sale/volvo/s90/110...
9,honda,['https://auto.ru/cars/used/sale/honda/shuttle...


### 2.2 Закачка карточек (расч 17 часов)

In [11]:
number_portion = 7 #номер порции для записи в файл (будет порядка 14 порций)
start_point = 6 # начало из списка брэндов
end_point = len(db_link) #len(db_link) #len(db_link) # конец из списка брэндов
db_car = pd.DataFrame()

for i in range(start_point, end_point): # цикл по брендам 
    brand = db_link['0'][i] # наименование бренда
    link = db_link['1'][i] #список ссылок на объявления i-ого бренда
    link = list(set(link.replace('[','').replace(']','').replace("'",'').split(','))) # всякие преобразования и удаления дубликатов
    number_link = len(link) # количество объявлений, должно совпасть с количеством ссылок
    #print(link, len(link))
    threaded_start = time.time()
    try:
        print('Start: '+ brand)
        one_brand = get_data(link)
        db_car = db_car.append(pd.DataFrame(one_brand), ignore_index=True)
        #time.sleep(0.5)
    except Exception as exc:
        print('%r generated an exception: %s' % (brand, exc))
              
    print("Load time:", time.time() - threaded_start, "; Brand:", brand, "; Кол-во ссылок:", number_link,"; Скачано:", len(one_brand))
    if (i%1 == 0)|(i == end_point - 1):  # разбиваем порции по 2 бренда или достигли конца списка (i%1 == 0)&(i > 0))
       # запись в файл
        file_name = './data/'+ str(number_portion)+'_cars'+'.csv'
        db_car.to_csv(file_name, sep=';', encoding='utf-8', index=False) 
        number_portion += 1
        db_car = pd.DataFrame()
#db_car

Start: mitsubishi
Не найден какой тэг: https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1106543513-0cb77985/
Не найден какой тэг: https://auto.ru/cars/used/sale/mitsubishi/outlander/1106165686-0d232386/
Не найден какой тэг: https://auto.ru/cars/used/sale/mitsubishi/outlander/1106295296-238deae7/
Load time: 3512.621456384659 ; Brand: mitsubishi ; Кол-во ссылок: 5815 ; Скачано: 5815
Start: skoda
Не найден какой тэг: https://auto.ru/cars/used/sale/skoda/kodiaq/1106523945-10bfbc0b/
Не найден какой тэг: https://auto.ru/cars/used/sale/skoda/octavia/1106353987-cc0c11e5/
Не найден какой тэг: https://auto.ru/cars/used/sale/skoda/fabia/1103996324-297a7224/
Load time: 4926.5326192379 ; Brand: skoda ; Кол-во ссылок: 7111 ; Скачано: 7111
Start: volvo
Load time: 1499.7502527236938 ; Brand: volvo ; Кол-во ссылок: 2318 ; Скачано: 2318
Start: honda
Load time: 1982.1426219940186 ; Brand: honda ; Кол-во ссылок: 3282 ; Скачано: 3282
Start: infiniti
Load time: 875.4835524559021 ; Brand: infiniti ; Ко

## 2.3 Дозакачка некоторых сломанных ссылок

In [51]:
db_link = pd.read_csv('./data/12 брендов по все россии/links_dop_1.csv', sep=';') 
db_link

,0,1
0,bmw,['https://auto.ru/cars/used/sale/bmw/i3/110600...
1,volkswagen,['https://auto.ru/cars/used/sale/volkswagen/go...
2,nissan,['https://auto.ru/cars/used/sale/nissan/leaf/1...
3,mercedes,['https://auto.ru/cars/used/sale/mercedes/eqv/...
4,toyota,['https://auto.ru/cars/used/sale/toyota/rav_4/...
5,audi,['https://auto.ru/cars/used/sale/audi/e_tron/1...
6,mitsubishi,['https://auto.ru/cars/used/sale/mitsubishi/mi...
7,honda,['https://auto.ru/cars/used/sale/honda/fit/110...
8,lexus,['https://auto.ru/cars/used/sale/lexus/ux/1105...


In [54]:
# уже совсем нет времени оформить в функцию, :(

number_portion = 14 #номер порции для записи в файл (будет порядка 14 порций)
start_point = 0 # начало из списка брэндов
end_point = len(db_link) #len(db_link) #len(db_link) # конец из списка брэндов
db_car = pd.DataFrame()

for i in range(start_point, end_point): # цикл по брендам 
    brand = db_link['0'][i] # наименование бренда
    link = db_link['1'][i] #список ссылок на объявления i-ого бренда
    link = list(set(link.replace('[','').replace(']','').replace("'",'').split(','))) # всякие преобразования и удаления дубликатов
    number_link = len(link) # количество объявлений, должно совпасть с количеством ссылок
    #print(link, len(link))
    threaded_start = time.time()
    try:
        print('Start: '+ brand)
        one_brand = get_data(link)
        db_car = db_car.append(pd.DataFrame(one_brand), ignore_index=True)
        #time.sleep(0.5)
    except Exception as exc:
        print('%r generated an exception: %s' % (brand, exc))
              
    print("Load time:", time.time() - threaded_start, "; Brand:", brand, "; Кол-во ссылок:", number_link,"; Скачано:", len(one_brand))
    if ((i%10 == 0)&(i > 0))|(i == end_point - 1):  # разбиваем порции по 2 бренда или достигли конца списка (i%1 == 0)&(i > 0))
       # запись в файл
        file_name = './data/'+ str(number_portion)+'_cars'+'.csv'
        db_car.to_csv(file_name, sep=';', encoding='utf-8', index=False) 
        number_portion += 1
        db_car = pd.DataFrame()
#db_car

Start: bmw
Load time: 32.710670948028564 ; Brand: bmw ; Кол-во ссылок: 57 ; Скачано: 57
Start: volkswagen
Load time: 12.79739236831665 ; Brand: volkswagen ; Кол-во ссылок: 18 ; Скачано: 18
Start: nissan
Load time: 119.77615642547607 ; Brand: nissan ; Кол-во ссылок: 222 ; Скачано: 222
Start: mercedes
Load time: 14.17813491821289 ; Brand: mercedes ; Кол-во ссылок: 21 ; Скачано: 21
Start: toyota
Load time: 9.285266876220703 ; Brand: toyota ; Кол-во ссылок: 11 ; Скачано: 11
Start: audi
Load time: 19.27053475379944 ; Brand: audi ; Кол-во ссылок: 32 ; Скачано: 32
Start: mitsubishi
Load time: 9.980623245239258 ; Brand: mitsubishi ; Кол-во ссылок: 13 ; Скачано: 13
Start: honda
Load time: 4.883499622344971 ; Brand: honda ; Кол-во ссылок: 4 ; Скачано: 4
Start: lexus
Load time: 4.479398965835571 ; Brand: lexus ; Кол-во ссылок: 1 ; Скачано: 1


In [29]:
db_car

""


## выяснилось что многих тегов нет и надо учиться обрабатывать строку, не теряя остальную информацию из объявления

# Вывод
1. Много данных парсится долго. 
2. На авто ру объявления со временем меняет структуру.
3. Некоторые теги не обязательны к заполнению (например description, engine)

# Что можно улучшить (но нет времени)
1. Оформить еще несколько функций
2. Придумать так чтобы при отсутствии тега строка продолжила заполняться имеющимися данными. Может есть вариант не оформлять каждый тег в try except
(если специалисты подскажут как это сделать, буду благодарна)

In [ ]:
sys.exit() 